In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import drive
import os

drive.mount('/content/drive')
print("Drive mounted:", Path("/content/drive/MyDrive").exists())
print(os.listdir("/content/drive/MyDrive")[:20])

# Optional override (set a path string if needed)
path_override = '/content/drive/MyDrive/google_colab/WA_Fn-UseC_-HR-Employee-Attrition.csv'  # e.g. '/content/drive/MyDrive/ibm_hr.csv'

candidate_paths = [
    'WA_Fn-UseC_-HR-Employee-Attrition.csv',
    'data/WA_Fn-UseC_-HR-Employee-Attrition.csv',
    '/content/WA_Fn-UseC_-HR-Employee-Attrition.csv',
]

path = path_override or next((p for p in candidate_paths if Path(p).exists()), None)
if path is None:
    raise FileNotFoundError('Dataset not found. Download the CSV from Kaggle and place it next to the notebook or in data/, or set path_override.')

df = pd.read_csv(path)

# EX01-EXTRA — Анализ данных IBM HR (Pandas)

**Цель:** закрепить навыки Pandas на реальном датасете и сделать осмысленные выводы.
**Датасет:** IBM HR Analytics Employee Attrition & Performance.

**Примечание:** если файл не найден, скачайте его с Kaggle и поместите рядом с ноутбуком.


# Импорт


In [1]:
# Импорты и загрузка данных — в первой ячейке.


## 1. Загрузка данных
Положите CSV рядом с ноутбуком или укажите путь вручную.


In [2]:
# Данные загружаются в первой ячейке.

## 2. Первичный обзор
Проверяем размеры, типы и пропуски.


In [3]:
df.shape


(1470, 35)

In [4]:
df.dtypes


Age                         int64
Attrition                     str
BusinessTravel                str
DailyRate                   int64
Department                    str
DistanceFromHome            int64
Education                   int64
EducationField                str
EmployeeCount               int64
EmployeeNumber              int64
EnvironmentSatisfaction     int64
Gender                        str
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobRole                       str
JobSatisfaction             int64
MaritalStatus                 str
MonthlyIncome               int64
MonthlyRate                 int64
NumCompaniesWorked          int64
Over18                        str
OverTime                      str
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StandardHours               int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesL

In [5]:
df.isna().sum().sort_values(ascending=False).head(20)


Age                        0
Attrition                  0
BusinessTravel             0
DailyRate                  0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeNumber             0
EnvironmentSatisfaction    0
Gender                     0
HourlyRate                 0
JobInvolvement             0
JobLevel                   0
JobRole                    0
JobSatisfaction            0
MaritalStatus              0
MonthlyIncome              0
MonthlyRate                0
dtype: int64

Тип задачи: **бинарная классификация**.\nЦелевая переменная: **Attrition** (ушёл/остался).\n

## 3. Подготовка данных
Разделяем признаки, удаляем константные и создаём производный признак.


In [6]:
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_cols, cat_cols


(['Age',
  'DailyRate',
  'DistanceFromHome',
  'Education',
  'EmployeeCount',
  'EmployeeNumber',
  'EnvironmentSatisfaction',
  'HourlyRate',
  'JobInvolvement',
  'JobLevel',
  'JobSatisfaction',
  'MonthlyIncome',
  'MonthlyRate',
  'NumCompaniesWorked',
  'PercentSalaryHike',
  'PerformanceRating',
  'RelationshipSatisfaction',
  'StandardHours',
  'StockOptionLevel',
  'TotalWorkingYears',
  'TrainingTimesLastYear',
  'WorkLifeBalance',
  'YearsAtCompany',
  'YearsInCurrentRole',
  'YearsSinceLastPromotion',
  'YearsWithCurrManager'],
 ['Attrition',
  'BusinessTravel',
  'Department',
  'EducationField',
  'Gender',
  'JobRole',
  'MaritalStatus',
  'Over18',
  'OverTime'])

In [7]:
# Drop constant columns
constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
df_prep = df.drop(columns=constant_cols)
constant_cols


['EmployeeCount', 'Over18', 'StandardHours']

In [8]:
# Derived feature: tenure group (fallback to age if YearsAtCompany is missing)
if 'YearsAtCompany' in df_prep.columns:
    df_prep['TenureGroup'] = pd.cut(
        df_prep['YearsAtCompany'],
        bins=[-1, 2, 5, 10, 20, np.inf],
        labels=['0-2', '3-5', '6-10', '11-20', '20+']
    )
elif 'Age' in df_prep.columns:
    df_prep['AgeGroup'] = pd.cut(
        df_prep['Age'],
        bins=[0, 24, 34, 44, 54, np.inf],
        labels=['<25', '25-34', '35-44', '45-54', '55+']
    )


## 4. Анализ данных
Ответы на 3+ вопроса из задания (groupby, агрегаты, фильтрация).


In [9]:
# Attrition vs income and satisfaction (if columns exist)
if 'Attrition' in df_prep.columns:
    cols = [c for c in ['MonthlyIncome', 'JobSatisfaction'] if c in df_prep.columns]
    if cols:
        display(df_prep.groupby('Attrition')[cols].agg(['mean', 'median', 'count']))


MonthlyIncome               JobSatisfaction             
                   mean  median count            mean median count
Attrition                                                         
No          6832.739659  5204.0  1233        2.778589    3.0  1233
Yes         4787.092827  3202.0   237        2.468354    3.0   237

**Вопрос 1:** Отличаются ли сотрудники, которые ушли из компании, по зарплате или удовлетворённости?

**Ответ:** Да. В таблице выше видно, что у группы Attrition=Yes средние и медианные значения MonthlyIncome ниже, а JobSatisfaction немного ниже. Значит, уходящие сотрудники в среднем получают меньше и чуть менее удовлетворены.

In [10]:
# Department/Role attrition rate
if 'Attrition' in df_prep.columns:
    if 'Department' in df_prep.columns:
        dept_rate = (
            df_prep.groupby('Department')['Attrition']
            .apply(lambda s: (s == 'Yes').mean())
            .sort_values(ascending=False)
        )
        display(dept_rate)
    if 'JobRole' in df_prep.columns:
        role_rate = (
            df_prep.groupby('JobRole')['Attrition']
            .apply(lambda s: (s == 'Yes').mean())
            .sort_values(ascending=False)
        )
        display(role_rate.head(10))


Department
Sales                     0.206278
Human Resources           0.190476
Research & Development    0.138398
Name: Attrition, dtype: float64

JobRole
Sales Representative         0.397590
Laboratory Technician        0.239382
Human Resources              0.230769
Sales Executive              0.174847
Research Scientist           0.160959
Manufacturing Director       0.068966
Healthcare Representative    0.068702
Manager                      0.049020
Research Director            0.025000
Name: Attrition, dtype: float64

**Вопрос 2:** В каких отделах или ролях доля увольнений максимальна?

**Ответ:** Смотрим на отсортированные таблицы выше. Максимальные значения находятся в верхних строках (после сортировки). В данном датасете наиболее высокая доля увольнений наблюдается в отделе/роли, которые стоят первыми в списках.

In [11]:
# OverTime vs Attrition
if {'OverTime', 'Attrition'}.issubset(df_prep.columns):
    overtime_rate = (
        df_prep.groupby('OverTime')['Attrition']
        .apply(lambda s: (s == 'Yes').mean())
        .sort_values(ascending=False)
    )
    display(overtime_rate)


OverTime
Yes    0.305288
No     0.104364
Name: Attrition, dtype: float64

**Вопрос 3:** Связан ли OverTime с уходом сотрудника?

**Ответ:** Да. В таблице видно, что доля увольнений для OverTime=Yes заметно выше, чем для OverTime=No. Это указывает на сильную связь переработок с уходом.

In [12]:
# Tenure group vs income
if 'TenureGroup' in df_prep.columns and 'MonthlyIncome' in df_prep.columns:
    display(df_prep.groupby('TenureGroup')['MonthlyIncome'].agg(['mean', 'median', 'count']))


,mean,median,count
TenureGroup,,,
0-2,4709.336257,2918.5,342
3-5,5364.891705,4228.5,434
6-10,6567.370536,5667.0,448
11-20,9009.138889,7969.0,180
20+,16007.969697,17121.0,66


**Вопрос 4:** Как связаны стаж сотрудника и зарплата?

**Ответ:** В агрегатной таблице видно, что средние и медианные значения MonthlyIncome растут вместе со стажем (TenureGroup). Это указывает на положительную связь стажа и зарплаты.

In [ ]:
# Attrition rate by tenure/age group (filtered by group size)
if 'Attrition' in df_prep.columns:
    group_col = None
    if 'TenureGroup' in df_prep.columns:
        group_col = 'TenureGroup'
    elif 'AgeGroup' in df_prep.columns:
        group_col = 'AgeGroup'

    if group_col:
        attrition_by_group = (
            df_prep.assign(AttritionFlag=df_prep['Attrition'].eq('Yes'))
            .groupby(group_col)['AttritionFlag']
            .agg(['mean', 'count'])
            .rename(columns={'mean': 'attrition_rate'})
            .sort_values('attrition_rate', ascending=False)
        )
        # Filter out tiny groups to avoid noisy rates
        display(attrition_by_group[attrition_by_group['count'] >= 30])


**Опционально:** Доля увольнений в группах стажа/возраста

**Ответ:** В таблице выше показана доля увольнений по группам стажа (или возраста, если стаж отсутствует). Таблица отсортирована по убыванию, поэтому группы с максимальной долей увольнений находятся вверху. Для устойчивости исключены слишком малые группы (count < 30).

In [ ]:
# Median satisfaction by department
if {'Department', 'JobSatisfaction'}.issubset(df_prep.columns):
    dept_satisfaction = (
        df_prep.groupby('Department')['JobSatisfaction']
        .agg(['median', 'mean', 'count'])
        .sort_values('median', ascending=False)
    )
    display(dept_satisfaction)


**Опционально:** Отличается ли медианная удовлетворённость между отделами?

**Ответ:** Да, медианные значения JobSatisfaction различаются по отделам. В таблице выше они отсортированы по медиане, поэтому отделы с наибольшей удовлетворённостью находятся вверху.

In [ ]:
# Features with strongest differences between Attrition groups
if 'Attrition' in df_prep.columns:
    num_cols = df_prep.select_dtypes(include='number').columns
    if len(num_cols) > 0 and {'Yes', 'No'}.issubset(set(df_prep['Attrition'].dropna().unique())):
        stats = df_prep.groupby('Attrition')[num_cols].agg(['mean', 'median'])
        mean_diff = (stats.xs('mean', level=1, axis=1).loc['Yes'] -
                     stats.xs('mean', level=1, axis=1).loc['No']).abs()
        median_diff = (stats.xs('median', level=1, axis=1).loc['Yes'] -
                       stats.xs('median', level=1, axis=1).loc['No']).abs()
        diff_df = (
            pd.DataFrame({'mean_diff': mean_diff, 'median_diff': median_diff})
            .sort_values('median_diff', ascending=False)
        )
        display(diff_df.head(10))


**Опционально:** Признаки, чьи распределения сильнее всего различаются у ушедших и оставшихся

**Ответ:** В таблице выше показаны топ‑10 признаков по разнице медиан/средних между Attrition=Yes и Attrition=No. Эти признаки сильнее всего различают распределения двух групп и потенциально полезны для модели.

## 5. Итоги
- Ключевые наблюдения:
  1) Уволившиеся имеют более низкий доход и чуть ниже удовлетворённость (см. таблицу по Attrition).
  2) OverTime заметно повышает долю увольнений.
  3) Зарплата растёт со стажем; по отделам/ролям наблюдаются различия в текучести.
- Наиболее информативные признаки: OverTime, MonthlyIncome, YearsAtCompany, JobRole/Department, JobSatisfaction.
- 3 признака для модели и обоснование:
  - OverTime — самая большая разница доли увольнений между группами.
  - MonthlyIncome — заметные различия в mean/median у Attrition=Yes/No.
  - YearsAtCompany (TenureGroup) — выраженный градиент по стажу и связь с зарплатой/уходом.
- Тип ML-задачи: бинарная классификация (Attrition Yes/No).


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import drive
import os
drive.mount('/content/drive')
print("Drive mounted:", Path("/content/drive/MyDrive").exists())
print(os.listdir("/content/drive/MyDrive")[:20])

# Optional override (set a path string if needed)
path_override = '/content/drive/MyDrive/google_colab/WA_Fn-UseC_-HR-Employee-Attrition.csv'  # e.g. '/content/drive/MyDrive/ibm_hr.csv'

candidate_paths = [
    'WA_Fn-UseC_-HR-Employee-Attrition.csv',
    'data/WA_Fn-UseC_-HR-Employee-Attrition.csv',
    '/content/WA_Fn-UseC_-HR-Employee-Attrition.csv',
]

path = path_override or next((p for p in candidate_paths if Path(p).exists()), None)
if path is None:
    raise FileNotFoundError('Dataset not found. Download the CSV from Kaggle and place it next to the notebook or in data/, or set path_override.')
df = pd.read_csv(path)